In [3]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ..

/home/ubuntu/AYP/code


In [7]:
!pip install gensim elasticsearch elasticsearch_dsl pymed

     |████████████████████████████████| 24.2 MB 58 kB/s s eta 0:00:01
  Using cached elasticsearch-7.8.0-py2.py3-none-any.whl (188 kB)
  Using cached elasticsearch_dsl-7.2.1-py2.py3-none-any.whl (53 kB)
  Using cached pymed-0.8.9-py3-none-any.whl (9.6 kB)
  Using cached smart_open-2.1.0.tar.gz (116 kB)
  Created wheel for smart-open: filename=smart_open-2.1.0-py3-none-any.whl size=110317 sha256=3765a075c5c9075e05ec028f6795f9f85bcccd838756f24557fbe819cd5c65bd
  Stored in directory: /home/ubuntu/.cache/pip/wheels/a4/9b/d5/85705a7ab783cd6f7bd718f01d3b1396272f30044e3c36401a
Successfully built smart-open


In [75]:
import os, re, sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from yuval_module.paper_clusterer import PaperClusterer
from yuval_module.paper_source import PaperSource

from sklearn.feature_extraction.text import TfidfVectorizer

import py_3.sim_matrix_3 as sim_matrix_3
import py_3.lr_model_3 as lr_model_3
import py_3.db_scan_3 as db_scan_3
import py_4.get_mesh_vec as get_mesh_vec
import py_4.get_cat_vec as get_cat_vec

import metric_eval_2

%matplotlib inline


import utils
from utils import PROJECT_ROOT, DATA_PATH

FILE = "enriched_labeled_dataset_large.json"

In [78]:
if os.path.exists(PROJECT_ROOT + DATA_PATH + FILE):
    print("READING FROM LOCAL")
    if FILE.split(".")[1] == "json":
        df = pd.read_json(PROJECT_ROOT + DATA_PATH + FILE)
    else:
        df = pd.read_csv(PROJECT_ROOT + DATA_PATH + FILE)
    ps = PaperSource()
else:
    print("PULLING FROM S3")
    ps = sim_matrix_3.load_dataset("enriched_labeled_dataset_large")
    df = ps.get_dataset()

# df.drop(columns=["ORG_STATE"],inplace=True)
# df.rename(columns={'ORG_STATE':'last_author_country'},inplace=True)

print("FILE PULLED")

PULLING FROM S3
FILE PULLED


In [79]:
df.columns

Index(['pmid', 'ORG_STATE', 'PI_IDS', 'last_author_name', 'authors', 'doi',
       'journal_name', 'language', 'last_author', 'last_author_country',
       'last_author_email', 'last_author_id', 'last_author_inst', 'mesh',
       'mesh_major', 'pub_year', 'title'],
      dtype='object')

In [14]:
df_auth = ps.add_processed_fields(df)

In [60]:
df_auth.columns

Index(['pmid', 'last_author_country', 'PI_IDS', 'last_author_name', 'authors',
       'doi', 'journal_name', 'language', 'last_author', 'last_author_email',
       'last_author_id', 'last_author_inst', 'mesh', 'mesh_major', 'pub_year',
       'title', 'mesh_clean', 'other_authors', 'inst_clean', 'email_clean'],
      dtype='object')

#### EXPLORING INSTITUTIONS

In [24]:
freq_inst = df_auth.groupby('last_author_inst').size().sort_values()

In [40]:
freq_inst.describe

count    2651.000000
mean       62.792908
std       286.544396
min         1.000000
25%         1.000000
50%         2.000000
75%        10.000000
max      4224.000000
dtype: float64

In [74]:
freq_inst[freq_inst > 4].index

Index(['VA Loma Linda Healthcare System', 'Southeastern University',
       'Southeast Louisiana Veterans Health Care System',
       'Florida Hospital Orlando', 'All India Institute of Medical Sciences',
       'Kenyatta National Hospital', 'White River Junction VA Medical Center',
       'William & Mary', 'Winthrop-University Hospital',
       'Centers for Disease Control and Prevention',
       ...
       'University of Michigan–Ann Arbor', 'Massachusetts General Hospital',
       'Harvard Medical School', 'University of Washington',
       'Stanford University', 'University of California, San Diego',
       'University of California, San Francisco', 'University of Pittsburgh',
       'Johns Hopkins University', 'University of Pennsylvania'],
      dtype='object', name='last_author_inst', length=985)

In [50]:
# Perc. without inst
df_auth['last_author_inst'].isna().sum() / len(df_auth['last_author_inst'])

0.14856093581369656

#### EXPLORING COUNTRY

In [70]:
country_stats = df.groupby('last_author_country').size().sort_values(ascending=False)

In [72]:
country_stats[:20]

last_author_country
United States     181452
China               1835
Georgia             1749
Canada              1446
India               1326
United Kingdom      1100
Jersey               812
Germany              501
Australia            439
Mexico               402
Japan                327
Italy                294
Lebanon              288
France               234
Netherlands          175
Brazil               160
Switzerland          150
South Africa         140
Israel               137
Sweden               137
dtype: int64

In [66]:
country_stats.describe()

count       122.000000
mean       1599.286885
std       16420.637743
min           1.000000
25%           3.000000
50%          11.500000
75%          55.750000
max      181452.000000
dtype: float64

In [80]:
state_stats = df.groupby('ORG_STATE').size().sort_values(ascending=False)

In [83]:
state_stats[:20]

ORG_STATE
CA    27540
MA    19741
NY    17378
PA    12529
TX    10878
IL     7628
NC     7450
MD     7335
MI     6480
OH     6163
GA     5153
WA     4855
FL     4613
MN     4543
TN     4407
CT     4291
MO     4189
WI     3932
IN     3009
CO     2872
dtype: int64

In [84]:
state_stats.describe()

count       60.000000
mean      3258.483333
std       5144.188821
min          3.000000
25%        294.750000
50%       1317.500000
75%       4320.000000
max      27540.000000
dtype: float64

#### Implementing get_vec.py

In [166]:
gcv = get_cat_vec.CatFeat(df)

In [167]:
ohe_inst = gcv.get_ohe_inst(df.last_author_inst)

In [172]:
ohe_country = gcv.get_ohe_country(df.last_author_country)

In [174]:
np.sum(ohe_country)

195113.0